<a href="https://colab.research.google.com/github/glamacles/notebooks/blob/main/gaussian_processes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction



- We're familiar with random variables
- For example a multivariate Gaussian:
$$\pmb{y} \sim \mathcal{N}(\pmb{\mu}, \Sigma)$$
$$ P(\pmb{y}) = (2 \pi)^{-n/2} |\Sigma|^{-1/2} \exp \left ( -\frac{1}{2} (\pmb{y} - \pmb{\mu})^T \Sigma^{-1} (\pmb{y} - \pmb{\mu}) \right ) $$
- Samples are points in $n$-dimensional space
- A 2D example:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix

x0 = np.array([0., 0.])
sigma = np.array([
    [2., -1],
    [-1, 2.]
])

ys = np.random.multivariate_normal(x0, sigma, 500)
plt.scatter(ys[:,0], ys[:,1])
plt.title('Samples from 2d Gaussian Distribution')
plt.grid(True, linestyle=':')
plt.xlabel('x')
plt.ylabel('y')

# Gaussian Processes


- A Gaussian process can be thought of as a generalization of a Gaussian distribution
- Instead of a distribution over points, it can be thought of as a distribution over functions
- That is, samples are functions instead of points!
- A Gaussian process is fully defined by a mean function $m(x)$ and a kernel $k(x,x')$
- Kernel represents relationship between "nearby" points
- For example, consider the Radial Basis Function (RBF) Kernel
 $$k(x, x') = \sigma^2 \exp \left ( -\frac{|| x - x' ||^2 }{ 2 l^2 }\right )$$
- Below we'll draw samples from a GP with $m(x)=0$ and an RBF kernel
- Nearby points in space are more highly correlated than those further apart


In [ ]:
# X coords
xs = np.linspace(0.,1.,100)

# Kernel Matrix
d = distance_matrix(xs[:,np.newaxis], xs[:,np.newaxis], p=2)
# Variance
sigma = 1.
# Length scale
l = 0.1
# Kernel
K = sigma**2 * np.exp(-d**2 / (2*l**2))


plt.title('Samples from a Gaussian Process')
# Plot realizations
ys = np.random.multivariate_normal(np.zeros_like(xs), K, 500)
for i in range(5):
    plt.plot(xs, ys[i], label=f'Sample {i}')
    plt.xlim([0.,1.])

plt.legend()
plt.xlabel('x')
plt.ylabel('y')


## Meaning of $m(x)$ and $k(x,x')$
- Mean function of a GP $m(x)$ defines the mean of the distribution of functions at a given point
- Kernel $k(x,x')$ can characterize the variance around the mean and "smoothness" of the samples
- In the RBF kernel these are controled by the parameters $\sigma$ and $l$ respectively
- Other choices for the kernel can encode other useful assumptions like periodicity

## Effect of RBF Kernel Paramters
- Below you can play around with the variance and length scale parameters in an RBF kernel to see how it affects the GP samples

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
from ipywidgets import interact

def plot_function(sigma, l):
    xs = np.linspace(0.,1.,100)

    # Kernel Matrix
    d = distance_matrix(xs[:,np.newaxis], xs[:,np.newaxis], p=2)
    # Kernel
    K = sigma**2 * np.exp(-d**2 / (2*l**2))

    ys = np.random.multivariate_normal(np.zeros_like(xs), K, 5)

    plt.clf()
    plt.title('Samples from GP with RBF Kernel')
    for i in range(5):
        plt.plot(xs, ys[i])
    plt.xlim([0.,1.])
    plt.ylim([-2, 2])
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()

interact(plot_function, sigma=(0.01, 1.), l=(0.01,0.5))


## Distribution of Random Variables in a GP
- A collection of random variables $f(\pmb{x}_1), \cdots, f(\pmb{x_n})$ is said to be drawn from a GP with mean $m(x)$ and covariance function $k(x,x')$ if they have the distribution
$$\pmb{f} \sim \mathcal{N} \left ( \pmb{m}, K \right ) $$

&emsp;&emsp; where
$$ \pmb{f} = [f(\pmb{x_1}), f(\pmb{x_2}), \cdots, f(\pmb{x_n})]^T$$
$$ \pmb{m} = [m(\pmb{x_1}), m(\pmb{x_2}), \cdots, m(\pmb{x_n})]^T$$
$$
K =
\begin{bmatrix}
k(\pmb{x_1}, \pmb{x_1}) & \cdots & k(\pmb{x_1}, \pmb{x_n}) \\
\vdots & \ddots & \vdots \\
k(\pmb{x_n}, \pmb{x_1}) & \cdots & k(\pmb{x_n}, \pmb{x_n})
\end{bmatrix} $$
- That is, if we take any set of points $\pmb{x_1}, \pmb{x_2}, \cdots, \pmb{x_n}$, the random variables corresponding to those points $f(\pmb{x_1}), f(\pmb{x_2}), \cdots, f(\pmb{x_n})$ follow a multivariate Gaussian distribution
- For example, if we look at two arbitrary points $x_1$ and $x_2$, $y$ values of our sample functions form a 2d Gaussian distribution

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact


def plot_function(i0, i1):
    plt.clf()

    plt.figure(figsize=(6,8))

    plt.subplot(2,1,1)
    plt.title('GP Samples')
    for i in range(25):
        plt.plot(xs, ys[i])
        plt.plot([xs[i0], xs[i1]], [ys[i,i0], ys[i,i1]], 'ko')
    plt.xlim([0,1.])


    plt.subplot(2,1,2)
    plt.title(f'Distribution of [f({xs[i0]:.2f}), f({xs[i1]:.2f})]')
    plt.scatter(ys[:,i0], ys[:,i1])
    plt.show()

interact(plot_function, i0=(0, 99), i1=(0, 99))


interactive(children=(IntSlider(value=49, description='i0', max=99), IntSlider(value=49, description='i1', max…

<function __main__.plot_function(i0, i1)>

# Gaussian Process Regression



## Scenario
- Suppose we have some noisy observations of an unknown function $F : \mathcal{R}^n \to \mathcal{R}$, and we assume
$$ \pmb{y} = F(X) + \pmb{\epsilon} $$
where, $\epsilon$ is independent and identically distributed (i.i.d.)
$$ \epsilon \sim \mathcal{N} \left ( \pmb{0}, \sigma^2 I \right ) $$
and the notation $F(X)$ means

$$ F(X) = [F(\pmb{x_1}), \cdots, F(\pmb{x_{N_0}})] $$
&emsp; &emsp; with
$$ X = [\pmb{x_1}, \cdots, \pmb{x_{N_0}}] $$
- The i.i.d. assumption can be relaxed, but this is a common assumption
- Now, suppose that we want to predict outputs of $F$ at unknown locations
$$ \pmb{y_*} = F(X^*) $$
where
$$X_* = [\pmb{x_1}_*, \cdots, \pmb{x_{N_1}}_*]$$



## Joint Distribution
- For vanilla Gaussian Process regression, we typically assume that the observations are generated by a GP with a zero mean and given covariance kernel $k(x,x')$
- Mean functions can also be inferred, however
- Under this assumption, the joint distribution of all points (observed $X$, and unobserved $X_*$) is Gaussian:
$$ \begin{bmatrix}
\pmb{y} \\
\pmb{y_*}
\end{bmatrix}
\sim
\mathcal{N} \left ( \pmb{0},
\begin{bmatrix}
 K(X,X) + \sigma^2 I & K(X,X_*)\\
 K(X_*, X) & K(X_*, X_*)
\end{bmatrix}
\right )$$
- The notation $K(X, X_*)$ is shorthand for the given covariance matrix
$$
K(X, X_*) = \begin{bmatrix}
 k(\pmb{x_1}, \pmb{{x_1}_*}) & \cdots & k(\pmb{x_1}, \pmb{{x_{N_1}}_*}) \\
 \vdots & \ddots & \vdots \\
 k(\pmb{x_{N_0}}, \pmb{{x_1}_*}) & \cdots & k(\pmb{x_{N_0}}, \pmb{{x_{N_1}}_*})
\end{bmatrix}
$$
- $K(X,X)$ and $K(X_*,X)$ are defined similarly

## Prediction: Computing the Conditional Distribution
- From here, can estimate the distribution for the unknown random variables $\pmb{y_*}$ at the unobserved points in the matrix $X^*$ by conditioning on the observations
- Here, we're using the nice properties of conditional distributions for Gaussians
- This yields an estimate of mean and covariance at our prediction points
$$ \pmb{y}^* \sim \mathcal{N}( \pmb{m}, P) $$
$$ \pmb{m} = K(X_*, X) \left ( K(X, X) + \sigma^2 I \right )^{-1} \pmb{y} $$
$$ P = K(X_*, X_*) - K(X_*, X) \left [ K(X, X) + \sigma^2 I \right ]^{-1} K(X, X_*)   $$
- If there is no noise, then the $GP$ regressor will match the observations exactly, but this can be numerically unstable

## Learning Hyperparamters
- Kernel hyperparameters such as the length scale and variance in the RBF can be set explicitly
- This is useful for some applications (e.g. when a GP is used as a prior)
- In some cases we would rather infer hyperparameters
- This is usally done via maximum likelihood estimation (MLE)
> MLE: Maximize the likelihood that the observations were drawn from a GP with the given hyperparamters
- In the setting of GP regression, assuming Gaussian noise, we already know the distribution of the data points given some paramters $\theta$
$$ P(\pmb{y}|\theta) \sim \mathcal{N}(\pmb{0}, K_{\theta}(X,X) + \sigma^2 I) $$
- The idea is to maximize this probability by varying $\theta$
- In practice, this is typically done by minimizing log likelihood w.r.t. hyperpramters $\theta$
$$ \log p \left ( \pmb{y} | X, \theta \right ) = -\frac{1}{2} \pmb{y}^T (K_{\theta}(X,X) + \sigma^2 I)^{-1} \pmb{y} - \frac{1}{2} | K_{\theta}(X,X) + \sigma^2 I | - \frac{n}{2} \log 2 \pi$$


# Example Implementation


- [Scikit-learn](https://scikit-learn.org) has an easy to use implementation
- Below we show a basic example
- Another good and very flexible implementation of GP regression is [GPytorch](https://gpytorch.ai/)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

X = np.linspace(0., 10., 1000).reshape(-1, 1)

# Function to gregress
y = np.squeeze(X * np.sin(X))

# Randomly choose some x-coordinates where we have observations
rng = np.random.RandomState(1)
training_indices = rng.choice(np.arange(y.size), size=6, replace=False)
X_train, y_train = X[training_indices], y[training_indices]

# Get noisy observations
noise_std = 0.75
y_train_noisy = y_train + rng.normal(loc=0.0, scale=noise_std, size=y_train.shape)

# Do the Gaussian Process regression using an RBF kernel
kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gaussian_process = GaussianProcessRegressor(
    kernel=kernel, alpha=noise_std**2, n_restarts_optimizer=9
)
gaussian_process.fit(X_train, y_train_noisy)

# Predict function and variance at unknown locations
mean_prediction, std_prediction = gaussian_process.predict(X, return_std=True)

plt.plot(X, y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
plt.errorbar(
    X_train,
    y_train_noisy,
    noise_std,
    linestyle="None",
    color="tab:blue",
    marker=".",
    markersize=10,
    label="Observations",
)
plt.plot(X, mean_prediction, label="Mean prediction")
plt.fill_between(
    X.ravel(),
    mean_prediction - 1.96 * std_prediction,
    mean_prediction + 1.96 * std_prediction,
    color="tab:orange",
    alpha=0.5,
    label=r"95% confidence interval",
)
plt.legend()
plt.xlim([0.,10.])
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
plt.title("Gaussian process regression on a noisy dataset")

## Kernels
- There are many choices of the covariance kernel for different applications
- One useful kernel is the [Matern kernel](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.kernels.Matern.html)
$$ k(x, x') =  \frac{1}{\Gamma(\nu)2^{\nu-1}}\Bigg(
\frac{\sqrt{2\nu}}{l} d(x , x')
\Bigg)^\nu K_\nu\Bigg(
\frac{\sqrt{2\nu}}{l} d(x , x' )\Bigg)
$$
- Here $\Gamma$ is the gamma function, and $K_{\nu}$ is the modified bessel function of the second time.
- For $\nu \to \infty$ this converges to the RBF kernel
- With right parameters, functions can be more "jagged" than for RBF kernel
- Another useful kernel is a [periodic kernel](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.kernels.ExpSineSquared.html)
- Could be useful for regression of seasonal / diurnal signals
- GPytorch implements a large number of covariance kernels and allows for custom [kernels](https://docs.gpytorch.ai/en/v1.11/kernels.html)
- GPytorch even allows for a learned neural network based covariance kernel

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ExpSineSquared


#length_scale = 1.0
#nu = 1.5
#kernel = Matern(length_scale=length_scale, nu=nu)

length_scale = 1.0  # This controls the length scale of the kernel
periodicity = 3.0  # This is the period of the kernel
kernel = ExpSineSquared(length_scale=length_scale, periodicity=periodicity)

gp = GaussianProcessRegressor(kernel=kernel)
X = np.linspace(0, 10, 100).reshape(-1, 1)
samples = gp.sample_y(X, n_samples=5, random_state=0)

# Plotting the samples
plt.figure(figsize=(10, 6))
for i, sample in enumerate(samples.T):
    plt.plot(X, sample, lw=2, label=f'Sample {i+1}')

plt.title('GP Samples with Periodic Kernel')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.xlim([0,10.])
plt.show()

# Multiple Output Dimensions
- Note that in the setting outlined here, the points we're predicting values at can be in any dimension as long as the output is scalar
- If you want to predict multiple outputs, you can use one GP per output dimension
$$
\pmb{y} = [{GP}_1(\pmb{x}), \cdots, {GP}_n(\pmb{x})]
$$
- Curse of dimensionality applies, so learning very high dimensional input spaces is difficult

# Efficiency and Scaling

---



# Efficient Inference

- Doing inference or training GP hyperparamters scales according to the number of training points. Consider computing the mean and variance of a single test point $\pmb{x_*}$


## Computing the mean
- After determining the hyperparamters, the vector  
$$ \pmb{\alpha} = \left ( K(X,X) + \sigma^2 \right )^{-1} \pmb{y}$$
can be precomputed, for example, using the Cholesky decomposition
- Then, computing the mean for a given test point $\pmb{x_*}$ can be computed by
$$ \pmb{m} = K(X,\pmb{x^*})^T \pmb{\alpha} $$

## Computing the variance

- Again, for a single test point $\pmb{x_*}$ computing the variance requires computing
$$ v(\pmb{x_*}) = K(\pmb{x_*}, X) \left ( K + \sigma^2 I \right ) ^{-1} K(X, \pmb{x_*}) $$
- This can be solved efficiently using the Cholesky decomposition in $\mathcal{O}(n^2)$


## Note on Cholesky Decomposition

- If a matrix $A$ is a real and positive-definite (like a covariance matrix), it can be decomposed as
$$A = L L^T$$
where $L$ is lower triangular and $L^T$ is upper triangular.
- Suppose we want to solve $$A\pmb{x} = L L^T \pmb{x} = \pmb{b}$$
- First, we can solve $L \pmb{y} = \pmb{b} $ by forward substitution.
- Then we can solve $L^T \pmb{x} = \pmb{y}$ by back substitution.
- This has computational complexity $\mathcal{O}(n^2)$



# Approximate GPs



- There are numerous methods for speeding up GP regression using approximate GPs
- One approach is to use inducing point methods
- These typically select a subset of the original training points, conditioning only on those points

# Applications in Glaciology



- GP's have many useful applications in Glaciology
- Can be used for interpolation of unknown variables
- GP's can be used as priors over functions for inverse problems (e.g. [this paper](https://www.sciencedirect.com/science/article/pii/S0021999122001577) on variational inference / other examples [here](https://arxiv.org/pdf/2112.13663))
- GP's can be used for emulation (e.g. [Edwards et al. 2021](https://research-repository.st-andrews.ac.uk/bitstream/handle/10023/24263/Edwards_2021_Nature_Projectedlandice_AAM.pdf?sequence=1&isAllowed=y)).
- Another cool example is using a GP emulator to help reconstruct ice sheet geometry at [125 ka](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019JF005237)
- For most practical use cases GP emulation is paired with some dimensionality reduction like principal components analysis (PCA)

# Example 1

- [Hill et al. 2024](https://essopenarchive.org/users/832963/articles/1226265-computationally-efficient-subglacial-drainage-modeling-using-gaussian-process-emulators-glads-gp-v1-0?commit=d091d843a1a3fc443c8b310c9f4c098c6656e12e) uses GPs to create an emulator for the GlaDS subglacial hydrology model
- The main use case is for Bayesian inference of unknown parameters in the hydrology model
- Conceptually, a set of parameters is sampled and GlaDS simulations are run for each sample
- Given the results, the principal components of the time / spatially dependent simulation outputs are computed
- Then, a GP is trained to map parameter inputs to coefficients of the principal component basis functions
- This is an efficient way to get spatially / time dependent outputs efficiently
- Note, some similarities do a Deep Operator
<div>
<img src="https://github.com/glamacles/notebooks/blob/main/images/Screenshot 2025-06-20 224428.png?raw=1" width="700"/>
</div>

## Example 2

- [Downs et al. 2022](https://www.cambridge.org/core/journals/journal-of-glaciology/article/inferring-timedependent-calving-dynamics-at-helheim-glacier/D678DCC319C0228A087701B315E4C971) uses a GP emulator to estimate uncertain model parameters for Helheim glacier
- Idea is to infer parameters that lead to best agreement between modeled / observed terminus position
- An ensemble of model runs is performed with different model parameters
- Then a GP is used to estimate mismatch between observations / model for arbitrary parameters based on this ensemble data
- GP fills in the gaps between unknown parameters and allows for fast MCMC sampling
- FIgure below shows an example of model parameters inferred from MCMC using the GP emulator
- This sampling would be intractable without emulation

<div>
<img src="https://github.com/glamacles/notebooks/blob/main/images/gaussian_processes/emulator.png?raw=1" width="700"/>
</div>